# CityBikes

1) Send a request to CityBikes for the city of your choice.

In [ ]:
import requests
import pandas
import json
from pandas import json_normalize

citybikes_url = "https://api.citybik.es/v2/networks"
response = requests.get(citybikes_url)

networks = response.json()["networks"]

osaka_network = None
for network in networks:
    if "docomo bike share" in network.get("name").lower():
        osaka_network = network
        break

network_id = osaka_network["id"]
stations_url = f"https://api.citybik.es/v2/networks/{network_id}"
response = requests.get(stations_url)

stations_data = response.json()["network"]["stations"]


The above script sends a request to CityBikes to retrieve the data for the stations in the city of Osaka, Japan. Upon exploring the structure of the API on CityBike's site map, it seems Osaska does not have only one Bike Share network for the entire city, but rather multiple depending on the Network Service Provider. In this case, the DOCOMO network was chosen and used as the Osaka bike share equivalent. The data retrieve request was run and stored in the JSON - "stations_data".

2) Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes).

In [ ]:
parsed_data = []

for station in stations_data:
    parsed_data.append({
        "name": station.get("name"),
        "latitude": station.get("latitude"),
        "longitude": station.get("longitude"),
        "free_bikes": station.get("free_bikes")
    })


This section of the script parses all the data from the JSON created previously (stations_data) to only show the requested information - latitude, longtitude, and number of bikes (listed as "free_bikes" in the CityBikes API). The "name" data was added in to help identify which station the information corresponds to.

3) Put your parsed results into a DataFrame.

In [ ]:
df = json_normalize(parsed_data)

print(f"\nParsed {len(df)} stations for Osaka (Docomo Bike Share):\n")
print(df.to_string)


This section of the script puts the parsed results into the Pandas DataFrame (line 1), using the "json_normalize" function to flatten the data into a nicer table to work with in the future. It also prints out the first 5 lines to visually preview the data to make sure the parsed information is as expected. However, as it is a simple print of the DataFrame contents, it is not as nicely formatted at this stage (and in particular, in a Colab notebook).

4a) Optional - Save JSON to local directory

In [ ]:
with open("osaka_docomo_stations.json", "w", encoding="utf-8") as f:
    json.dump(stations_data, f, indent=2, ensure_ascii=False)

print("JSON file saved.")

4b) Optional - Save as CSV to local directory

In [ ]:
df.to_csv("osaka_docomo_stations.csv", index=False, encoding="utf-8-sig")

print("CSV file saved.")

*For both the above optional scripts, utf-8 encoding is used since we are working with Osaka bike stations. Most station names are in Japanese characters.